In [14]:

%%capture
!pip install git+https://github.com/neuml/txtai
!pip install datasets
!pip install git+https://github.com/neuml/txtai#egg=txtai[pipeline]
!pip install transformers datasets

# Get test data
!wget -N https://github.com/neuml/txtai/releases/download/v6.2.0/tests.tar.gz
!tar -xvzf tests.tar.gz

# Install NLTK
import nltk
nltk.download('punkt')


In [30]:
from google.colab import files
upload = files.upload()

Saving Summary of Right Classification of NYP Systems.docx to Summary of Right Classification of NYP Systems.docx


In [46]:
%%capture
from txtai.pipeline import Textractor
# Create textractor model
textractor = Textractor()

In [ ]:
textractor = Textractor(paragraphs=True)
for paragraph in textractor("Guide to Right Classification - NYP ppt v190801.pdf"):
  print(paragraph, "\n----")

INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


Restricted 
----
Sharing on: 
----
2019 Guide to Right Classification 
----
Restricted 
----
CLASSIFIED 
----
TOP SECRET
Exceptionally grave damage 
----
to national security 
----
SECRET
Serious damage to national 
----
interests/ security 
----
CONFIDENTIAL
Serious damage to an 
----
agency; some damage to 
national interests 
----
RESTRICTED
Some damage to an agency 
----
UNCLASSIFIED
Residual of information that 
----
is not classified as RCST 
----
Official Information:
”All info that public officers acquire (except that which
is already openly or publicly available), generate or
cause to be generated in the course of their duties” 
----
Classified Information:
”Official information that has been assigned a security
classification, based on the potential damage or
negative consequence of unauthorised disclosure” 
----
Source: From ISD Green Book 
----
OLD RCST Framework: does not define UNCLASSIFIED;
data was over-classified to RESTRICTED or CONFIDENTIAL 
----
Restricted
Right Cla

### **TXT File**

In [17]:
from google.colab import files
from txtai.embeddings import Embeddings
from txtai.pipeline import Textractor, Similarity

def upload_file():
    uploaded = files.upload()
    file_name = next(iter(uploaded))
    content = uploaded[file_name].decode("utf-8")  # Decode bytes to string
    return file_name, content

def read_content(content):
    lines = content.splitlines()
    return [{"text": line.strip()} for line in lines if line.strip()]

def stream_from_content(data, field, limit):
    index = 0
    for item in data:
        yield (index, item[field], None)
        index += 1
        if index >= limit:
            break

# Upload and read file
file_name, content = upload_file()
custom_data = read_content(content)

# Create embeddings model, backed by sentence-transformers & transformers, enable content storage
embeddings = Embeddings({"path": "sentence-transformers/paraphrase-MiniLM-L3-v2", "content": True})
embeddings.index(stream_from_content(custom_data, "text", len(custom_data)))

# Create similarity instance for re-ranking
similarity = Similarity("valhalla/distilbart-mnli-12-3")

# Define search functions
def search(query):
    return [(result["score"], result["text"]) for result in embeddings.search(query, limit=50)]

def ranksearch(query):
    results = [text for _, text in search(query)]
    return [(score, results[x]) for x, score in similarity(query, results)]

# Example search
query = "president of Singapore"
search_results = search(query)
print("Search results:")
for score, text in search_results:
    print(f"Score: {score:.4f}")
    print(f"Text: {text}\n")

# Ranked search results
ranked_results = ranksearch(query)
print("Ranked search results:")
for score, text in ranked_results:
    print(f"Score: {score:.4f}")
    print(f"Text: {text}\n")


Saving sgpresident.txt to sgpresident (1).txt
Search results:
Score: 1.0000
Text: President of Singapore

Score: 0.7627
Text: The president of the Republic of Singapore is the head of state of Singapore. The president represents the country in official diplomatic functions and possesses certain executive powers over the Government of Singapore, including the control of the national reserves and the ability to revoke and appoint public service appointments. The president also holds the prerogative to grant pardons.

Score: 0.6855
Text: https://en.wikipedia.org/wiki/President_of_Singapore

Score: 0.6232
Text: After Singapore achieved full internal self-governance from the British Empire in 1959, the ceremonial office of the Yang di-Pertuan Negara (lit. "Lord of the State") was created. The office was later succeeded by the President of Singapore after Singapore's independence in 1965. The initial role of the president was largely ceremonial and symbolic, carrying residual powers, but the

### **CSV FILE**

In [28]:
from google.colab import files
from txtai.embeddings import Embeddings
from txtai.pipeline import Similarity

# Function to upload a file and return its content
def upload_file():
    uploaded = files.upload()
    file_name = next(iter(uploaded))
    with open(file_name, "r") as file:
        content = file.readlines()
    return file_name, content

# Function to read content from lines and format it as a list of dictionaries
def read_content(lines):
    return [{"text": line.strip()} for line in lines if line.strip()]

# Function to stream data for indexing
def stream_from_content(data, field, limit):
    index = 0
    for item in data:
        yield (index, item[field], None)
        index += 1
        if index >= limit:
            break

# Upload and read file
file_name, content = upload_file()
custom_data = read_content(content)

# Create embeddings model, backed by sentence-transformers & transformers, enable content storage
embeddings = Embeddings({"path": "sentence-transformers/paraphrase-MiniLM-L3-v2", "content": True})
embeddings.index(stream_from_content(custom_data, "text", len(custom_data)))

# Create similarity instance for re-ranking
similarity = Similarity("valhalla/distilbart-mnli-12-3")

# Define search functions
def search(query):
    return [(result["score"], result["text"]) for result in embeddings.search(query, limit=50)]

def ranksearch(query):
    results = [text for _, text in search(query)]
    return [(score, results[x]) for x, score in similarity(query, results)]

# Example search
query = "Sensitive"
print("Search results:")
for score, text in search(query):
    print(f"Score: {score:.4f}, Text: {text}")

print("\nRanked search results:")
for score, text in ranksearch(query):
    print(f"Score: {score:.4f}, Text: {text}")


Saving FAQ_Chatbot.csv to FAQ_Chatbot.csv
Search results:
Score: 0.5405, Text: What kind of information is considered sensitive?,"Sensitive information typically includes personal data such as addresses, phone numbers, social security numbers, credit card details, and other private information."
Score: 0.4090, Text: What is considered a non-sensitive email?,"A non-sensitive email contains information that does not include personal data, financial details, or any confidential information. It is generally safe to share publicly."
Score: 0.2858, Text: Is my email content secure?,"Yes, your email content is processed securely. We follow strict data protection protocols to ensure your information remains private and secure."
Score: 0.1700, Text: What is this chatbot for?,This chatbot helps you classify the content of your emails before you send them. It can determine if your email contains sensitive or non-sensitive information and whether it is private or public.
Score: 0.1240, Text: What 

### **PDF FILE**

In [29]:
from google.colab import files
from txtai.embeddings import Embeddings
from txtai.pipeline import Textractor, Similarity

# Function to upload a file and return its content
def upload_file():
    uploaded = files.upload()
    file_name = next(iter(uploaded))
    return file_name

# Function to clean and format extracted PDF text
def clean_pdf_text(pdf_text):
    cleaned_text = []
    for paragraph in pdf_text:
        # Example cleanup: Remove extra whitespace and symbols
        cleaned_paragraph = paragraph.strip().replace("\n", " ").replace("\t", " ")
        if cleaned_paragraph:
            cleaned_text.append({"text": cleaned_paragraph})
    return cleaned_text

# Function to stream data for indexing
def stream_from_content(data):
    for i, item in enumerate(data):
        yield (i, item["text"], None)

# Upload PDF file
file_name = upload_file()

# Create textractor model with paragraphs=True
textractor = Textractor(paragraphs=True)

# Extract text from PDF using textractor
pdf_text = textractor(file_name)

# Clean and format PDF text for indexing
custom_data = clean_pdf_text(pdf_text)

# Create embeddings model, backed by sentence-transformers & transformers, enable content storage
embeddings = Embeddings({"path": "sentence-transformers/paraphrase-MiniLM-L3-v2", "content": True})
embeddings.index(stream_from_content(custom_data))

# Create similarity instance for re-ranking
similarity = Similarity("valhalla/distilbart-mnli-12-3")

# Define search functions
def search(query):
    return [(result["score"], result["text"]) for result in embeddings.search(query, limit=50)]

def ranksearch(query):
    results = [text for _, text in search(query)]
    return [(score, results[x]) for x, score in similarity(query, results)]

# Example search
query = "Sensitive"
print("Search results:")
for score, text in search(query):
    print(f"Score: {score:.4f}, Text: {text}")

print("\nRanked search results:")
for score, text in ranksearch(query):
    print(f"Score: {score:.4f}, Text: {text}")


Saving Guide to Right Classification - NYP ppt v190801.pdf to Guide to Right Classification - NYP ppt v190801 (1).pdf
Search results:
Score: 0.8418, Text: Non Sensitive
Score: 0.8365, Text: Sensitive High
Score: 0.8365, Text: Sensitive High
Score: 0.7695, Text: Sensitive Normal
Score: 0.5493, Text: Sensitive Normal Some damage to  individual or business, if  leaked.
Score: 0.4773, Text: SENSITIVITY  Categorisation
Score: 0.4773, Text: SENSITIVITY  Categorisation
Score: 0.4773, Text: SENSITIVITY  Categorisation
Score: 0.4772, Text: Sensitive High Serious damage to an  individual or business  (e.g. incriminating  information), if leaked.
Score: 0.4397, Text: CONFIDENTIAL Serious damage to an
Score: 0.3749, Text: Non-Sensitive  Negligible or no damage  to an individual or  business, if divulged.
Score: 0.3724, Text: Restricted Right Classification Guide: Part II – Sensitivity Categorisation The Information Sensitivity Framework (ISF) considers the impact of unauthorised  disclosure of ent

**Word Doc**


In [47]:
from google.colab import files
from txtai.embeddings import Embeddings
from txtai.pipeline import Textractor, Similarity

# Function to upload a file and return its content
def upload_file():
    uploaded = files.upload()
    file_name = next(iter(uploaded))
    return file_name

# Function to clean and format extracted text
def clean_text(raw_text):
    # Example cleanup: Remove extra whitespace and symbols
    cleaned_text = raw_text.strip().replace("\n", " ").replace("\t", " ")
    return {"text": cleaned_text} if cleaned_text else None

# Function to stream data for indexing
def stream_from_content(data):
    for i, item in enumerate(data):
        yield (i, item["text"], None)

# Function to format search results nicely
def format_search_results(results):
    formatted_results = []
    for idx, result in enumerate(results, start=1):
        formatted_results.append(f"Result {idx}:")
        formatted_results.append(f"Score: {result['score']:.4f}")
        formatted_results.append(f"Text: {result['text']}")
        formatted_results.append("\n")
    return "\n".join(formatted_results)

# Main function for processing uploaded DOCX file
def process_docx_file(file_name):
    # Check if the file is a DOCX file
    if file_name.endswith(".docx"):
        # Create textractor model with paragraphs=True to extract paragraphs from DOCX
        textractor = Textractor(paragraphs=True)

        # Extract text from DOCX using textractor
        docx_text = textractor(file_name)

        # Clean and format extracted text for indexing
        custom_data = [clean_text(paragraph) for paragraph in docx_text if clean_text(paragraph)]

        # Create embeddings model, backed by sentence-transformers, enable content storage
        embeddings = Embeddings({"path": "sentence-transformers/paraphrase-MiniLM-L3-v2", "content": True})
        embeddings.index(stream_from_content(custom_data))

        # Define search function
        def search(query):
            results = embeddings.search(query, limit=50)
            return [{"score": result["score"], "text": result["text"]} for result in results]

        # Example search
        query = "Sensitive"
        results = search(query)

        # Format and print search results
        print("Search results:")
        formatted_results = format_search_results(results)
        print(formatted_results)

    else:
        print("Unsupported file format. Please upload a .docx file.")

# Upload DOCX file
file_name = upload_file()

# Process the uploaded DOCX file
process_docx_file(file_name)


Saving Summary of Right Classification of NYP Systems.docx to Summary of Right Classification of NYP Systems (1).docx
Search results:
Result 1:
Score: 0.3113
Text: Official (Closed) and Non-Sensitive NYP Systems Security Classification and Sensitivity Categorisation


Result 2:
Score: 0.1147
Text: |No.|System Name|Info Security Classification|Sensitivity Categorisation| |---|---|---|---| |1|ACI Website|Official Open|Non-Sensitive| |2|Analytics in Education (AIE)|Official Closed|Sensitive (Normal)| |3|Alumni Information Management System (AIMS)|Official Closed|Sensitive (Normal)| |4|Learning Management System|Official Closed|Sensitive (Normal)| |5|CET Revenue System|Official Closed|Sensitive (Normal)| |6|Clinical Funding Administration System (CFAS)|Official Closed|Sensitive (Normal)| |7|Co-Curricular Activities Points System (CCAPS)|Official Closed|Non-Sensitive| |8|Customer Relationship Mgmt System|Official Closed|Non-Sensitive| |9|Declaration for Conflicit of Interest in Academic Mat

### **PPT**

In [45]:
from google.colab import files
from txtai.embeddings import Embeddings
from txtai.pipeline import Textractor, Similarity

# Function to upload a file and return its content
def upload_file():
    uploaded = files.upload()
    file_name = next(iter(uploaded))
    return file_name

# Function to clean and format extracted text
def clean_text(raw_text):
    # Example cleanup: Remove extra whitespace and symbols
    cleaned_text = raw_text.strip().replace("\n", " ").replace("\t", " ")
    return {"text": cleaned_text} if cleaned_text else None

# Function to stream data for indexing
def stream_from_content(data):
    for i, item in enumerate(data):
        yield (i, item["text"], None)

# Function to format search results nicely
def format_search_results(results):
    formatted_results = []
    for idx, result in enumerate(results, start=1):
        formatted_results.append(f"Result {idx}:")
        formatted_results.append(f"Score: {result['score']:.4f}")
        formatted_results.append(f"Text: {result['text']}")
        formatted_results.append("\n")
    return "\n".join(formatted_results)

# Main function for processing uploaded PPTX file
def process_docx_file(file_name):
    # Check if the file is a DOCX file
    if file_name.endswith(".pptx"):
        # Create textractor model with paragraphs=True to extract paragraphs from PPTX
        textractor = Textractor(paragraphs=True)

        # Extract text from DOCX using textractor
        docx_text = textractor(file_name)

        # Clean and format extracted text for indexing
        custom_data = [clean_text(paragraph) for paragraph in docx_text if clean_text(paragraph)]

        # Create embeddings model, backed by sentence-transformers, enable content storage
        embeddings = Embeddings({"path": "sentence-transformers/paraphrase-MiniLM-L3-v2", "content": True})
        embeddings.index(stream_from_content(custom_data))

        # Define search function
        def search(query):
            results = embeddings.search(query, limit=50)
            return [{"score": result["score"], "text": result["text"]} for result in results]

        # Example search
        query = "Sensitive"
        results = search(query)

        # Format and print search results
        print("Search results:")
        formatted_results = format_search_results(results)
        print(formatted_results)

    else:
        print("Unsupported file format. Please upload a .pptx file.")

# Upload PPTX file
file_name = upload_file()

# Process the uploaded PPTX file
process_docx_file(file_name)


Saving CNC - Staff Induction 2024.pptx to CNC - Staff Induction 2024.pptx
Search results:
Result 1:
Score: 0.5687
Text: Official (Closed) and Sensitive-Normal


Result 2:
Score: 0.5687
Text: Official (Closed) and Sensitive-Normal


Result 3:
Score: 0.5687
Text: Official (Closed) and Sensitive-Normal


Result 4:
Score: 0.5687
Text: Official (Closed) and Sensitive-Normal


Result 5:
Score: 0.5687
Text: Official (Closed) and Sensitive-Normal


Result 6:
Score: 0.5687
Text: Official (Closed) and Sensitive-Normal


Result 7:
Score: 0.5687
Text: Official (Closed) and Sensitive-Normal


Result 8:
Score: 0.5687
Text: Official (Closed) and Sensitive-Normal


Result 9:
Score: 0.5687
Text: Official (Closed) and Sensitive-Normal


Result 10:
Score: 0.5318
Text: Official (Closed) and Sensitive-Normal Understanding the Sensitivity Levels Sensitivity levels measure the damage done to an entity (individual and/or business)


Result 11:
Score: 0.4322
Text: Official (Closed) and Sensitive-Normal Data Cl